In [1]:
import pandas as pd
import numpy as np

In [38]:
paths = ["C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/1_Eventos.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/2_MDB.xlsx",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/3_Dependientes.xlsx",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/4_edformal.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/5_Experiencia.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/6_idiomas.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/7_Intereses_carrera.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/8_software_skills.csv"
         ]

In [59]:
Eventos = pd.read_csv(paths[0], skiprows=1)
MDB = pd.read_excel(paths[1], skiprows=2)
Depends = pd.read_excel(paths[2], skiprows=2)
Education = pd.read_csv(paths[3], skiprows=1)
Exp = pd.read_csv(paths[4], skiprows=1)
Languages = pd.read_csv(paths[5], skiprows=1)
Interest = pd.read_csv(paths[6], skiprows=1)
Software = pd.read_csv(paths[7], skiprows=1)

In [41]:
def filter_events(df):
    # Filtrar registros según las condiciones dadas
    filtered_df = df.loc[
        (df['event'] == 'TERMINATION') &
        (df['eventReason'] == 'Retiro Voluntario')
    ]

    # Seleccionar columnas de interés
    columns_to_keep = ['eventCode', 'event', 'eventReasonCode', 'eventReason',
                       'effectiveDate', 'personId', 'termination detail']
    filtered_df = filtered_df[columns_to_keep]

    # Eliminar registros duplicados
    filtered_df = filtered_df.drop_duplicates()

    return filtered_df

In [54]:
def filter_depends(df):
    columns_to_keep = ['PERSON ID', 'DATE OF BIRTH','RELATIONSHIP']
    dfDep = df[columns_to_keep]

    # Eliminar los NaN
    dfDep.dropna(inplace = True)

    # Obtener conteo de dependientes de cada PersonId
    dfDep = dfDep.groupby('PERSON ID')['RELATIONSHIP'].count().reset_index()
    dfDep.rename(columns={'RELATIONSHIP': 'DEPENDIENTES'}, inplace=True)

    return dfDep

In [56]:
def filter_mdb(df):
    # Filtrar registros según las condiciones dadas
    dfMDB = df.loc[
        ((df['EMPLOYEE STATUS'] == 'TERMINATED') | (df['EMPLOYEE STATUS'] == 'ACTIVE')) &
        (df['COUNTRY'] == 'COL')
    ]

    # Eliminar registros duplicados
    dfMDB = dfMDB.drop_duplicates()

    return dfMDB

In [33]:
def filter_exp(df):

    # Quitar fechas erradas, volverlas NaN (ej: año 1810)
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
    df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')

    # Eliminar los NaN
    df.dropna(subset=['Start Date', 'End Date'], inplace=True)

    # Convertir en formato fecha
    df['Start Date']  = pd.to_datetime(df['Start Date'])
    df['End Date']  = pd.to_datetime(df['End Date'])

    # Obtener años de permanencia en cada experiencia
    df['Years_Exp'] = (df['End Date'] - df['Start Date']).dt.days / 365

    # Obtener años totales de experiencia de cada colaborador
    dfExp = df.groupby('Person ID External')['Years_Exp'].sum().reset_index()

    return dfExp

In [86]:
def filter_education (df):

    # Seleccionar las variables de interes
    dfEducation = df[['Person ID External', 'Academic degree']]

    # Dicotomizar variables
    dfEducation = pd.get_dummies(dfEducation, columns=['Academic degree'], drop_first=True, prefix=['Academic degree'])

    # Renombrar columnas
    dfEducation.rename(columns={"Person ID External": 'PersonID', 
                          "Academic degree_BACHELOR'S": "Bachelor",
                          "Academic degree_DOCTORATE'S": "Doctorate", 
                          "Academic degree_ELEMENTARY SCHOOL": "Elementary_school",
                          "Academic degree_HIGH SCHOOL": "High_school", 
                          "Academic degree_MASTER'S": "Master",
                          "Academic degree_SPECIALIZATION (COL)": "Specialization",
                          "Academic degree_TECHNICAL (COL)": "Technical"}, inplace=True)
    

    # Quitar duplicados por medio de agrupación
    dfEducation = dfEducation.groupby("PersonID").any().reset_index()

    return dfEducation

In [126]:
def filter_lang (df):

    # Seleccionar variables de interés
    df_Lang = df[['Person ID External', 'Language/Dialect' ]]

    # Dicotomizar variables
    df_Lang = pd.get_dummies(df_Lang, columns=['Language/Dialect'], drop_first=True, prefix=['Language/Dialect'])

    # Conteo de todos los lenguajes por colaborador
    df_Lang['Languages'] = df_Lang.iloc[:, 1:].sum(axis=1)
    df_Lang = df_Lang[['Person ID External', 'Languages']]
    return df_Lang

In [147]:
def filter_interest (df):

    # Seleccionar variables de interés
    dfInter = df[['Person ID External', 'Department/Team interested in']]

    # Eliminar registros que no tienen un area de interes
    dfInter = dfInter.dropna(subset=['Department/Team interested in'])

    # Agrupar todas las areas de interes en un solo registro
    dfInter = dfInter.groupby("Person ID External")["Department/Team interested in"].apply(lambda x: " / ".join(x)).reset_index()
    return dfInter

In [152]:
def filter_software (df):

    # Seleccionar variables de interés
    dfSoft = df[['Person ID External', 'Application']]

    # Eliminar registros que no tienen una skill
    dfSoft = dfSoft.dropna(subset=['Application'])

    # Agrupar todos los skills en un solo registro
    dfSoft = dfSoft.groupby("Person ID External")["Application"].apply(lambda x: " / ".join(x)).reset_index()
    return dfSoft